In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json
import requests

import pymongo
from pymongo import MongoClient
from pprint import pprint

# PyMongo Setup and Querying

In [4]:
client = MongoClient()
db = client['DS4300']
yelp_collection = db['Yelp']

In [7]:
# Example document
pprint(yelp_collection.find_one())

{'_id': ObjectId('622198b2b5127b859164a21e'),
 'address': '921 Pearl St',
 'attributes': {'Alcohol': "'beer_and_wine'",
                'Ambience': "{'touristy': False, 'hipster': False, 'romantic': "
                            "False, 'divey': False, 'intimate': False, "
                            "'trendy': False, 'upscale': False, 'classy': "
                            "False, 'casual': True}",
                'BikeParking': 'True',
                'BusinessAcceptsBitcoin': 'False',
                'BusinessAcceptsCreditCards': 'True',
                'BusinessParking': "{'garage': False, 'street': True, "
                                   "'validated': False, 'lot': False, 'valet': "
                                   'False}',
                'Caters': 'True',
                'DogsAllowed': 'False',
                'GoodForMeal': "{'dessert': False, 'latenight': False, "
                               "'lunch': False, 'dinner': False, 'brunch': "
                              

#### Q1. How many establishments have 5 stars?

In [52]:
num_5_stars = yelp_collection.find({'stars': 5})
len(list(num_5_stars))

19953

#### Q2. Which establishment has the least stars?

In [53]:
least_stars = yelp_collection.find_one(sort=[('stars', 1)])
least_stars

{'_id': ObjectId('622198b2b5127b859164a251'),
 'business_id': 'w4qVflIAbdklzG3mnKmQsg',
 'name': 'Magical Pizza Express',
 'address': '3262 Vineland Rd, Ste 106',
 'city': 'Orlando',
 'state': 'FL',
 'postal_code': '32811',
 'latitude': 28.5205,
 'longitude': -81.44395,
 'stars': 1,
 'review_count': 8,
 'is_open': 1,
 'attributes': {'RestaurantsAttire': "u'casual'",
  'Alcohol': "u'none'",
  'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': False}"},
 'categories': 'Italian, Restaurants',
 'hours': None}

#### Q3. What is the count of establishments for each star rating?

In [54]:
star_counts = yelp_collection.aggregate(
        [{"$group" : 
            {"_id" : "$stars", 
            "num_establishments" : {"$sum" : 1}
         }},
         {'$sort' : 
            {'_id' : 1}}
    ])
    
for i in star_counts:
    print(i)

{'_id': 1, 'num_establishments': 1686}
{'_id': 1.5, 'num_establishments': 4157}
{'_id': 2, 'num_establishments': 8523}
{'_id': 2.5, 'num_establishments': 13720}
{'_id': 3, 'num_establishments': 21583}
{'_id': 3.5, 'num_establishments': 28835}
{'_id': 4, 'num_establishments': 34056}
{'_id': 4.5, 'num_establishments': 28072}
{'_id': 5, 'num_establishments': 19953}


#### Q4. 